In [ ]:
# importing libraries
from utils import (
     prompts,
     llm,    
)
from core import hybrid_search
from utils.helpers import format_search_results, re_rank_docs
from ddgs import DDGS 
from IPython.display import Markdown, display

In [ ]:
# Extracting relevant documents from the vector database 
# Note: we are using qdrant vector db, which we already created for our UAE Legal RAG usecase
query = "what is the Managing director of Dubai Electricity Water Authority(DEWA)?"
retrivedDocs = hybrid_search.retreive_relevant_docs(query)

In [ ]:
# we will make use of decision prompt to decide if we have relevant context or not
# it returns 'Yes' if the context has an answer else it returns 'No' as answer
decision_prompt = prompts.decision_prompt(query, retrivedDocs)
has_answer, response_time = llm.generate_response( query=query, 
                                                   retrieved_results=retrivedDocs, 
                                                   prompt_template=decision_prompt,
                                                   model_name=llm.GROQ_MODEL2
                                                 )       

print(f'has_answer: {has_answer}, response time (in millisec): {response_time}')

In [ ]:
# Based on the decision prompt result, we either generate response from context or search online using duckduckgo-search
if has_answer == 'YES':
    print('Context can answer the question, generating response...')
    response_from_context = True
else:
    print('Context cannot answer the question, searching online for answer...')
    response_from_context = False
    response_from_ddgs = DDGS().text(query, max_results=5)
    sorted_docs = re_rank_docs(response_from_ddgs, query)
    formatted_results = format_search_results(sorted_docs)
    
retrieved_results = retrivedDocs if response_from_context else formatted_results

# Common code for generating the response
response, response_time = llm.generate_response(
    query=query,
    retrieved_results=retrieved_results,
    prompt_template=prompts.response_generation(query, retrieved_results),
    model_name=llm.GROQ_MODEL
)

print('Response:')
display(Markdown(response))
